<a href="https://colab.research.google.com/github/joshuaidumijie/Hamoye-Internship-2020/blob/master/Hamoye_Stage_A_by_Joshua_Idumijie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Quiz's Code are towards the end part of the code with the appropriate heading to identify it.

##Stage A Assignment by Joshua Idumijie
###The Objective of this Project is to determine:
+ Explore the Operating Cost of the Individual Power Plant
+ Investigate the impact of fuel cost on the viability of the different generation source
+ Highlight the competitiveness of Renewable Electricity and
+ Visualise the generation mix of different utilities eveloved over time.

## Some Background Information.
### From exploring the data manually and doing some research into the Energy field, I came up with the following milestones to achieve the aforementioned goals.


*   To confirm the operating cost, the the data set of all the costs from the dataset((fuel_cost_per_unit_burned,Fuel_cost_per_unit_delivered, fuel_cost_per_mmbtu all vs plant_name_ferc1))  and then compared to to each firm(over 2000 of them, not sure how to go ahead with this yet).
*   The costs vs the type of fuel will help determine the impact of fuel type on cost of operation. (fuel_cost_per_unit_burned,Fuel_cost_per_unit_delivered, fuel_cost_per_mmbtu all vs Fuel_type_code_pudl) 

*   The Renewable fuel type will be highlighted to confirmed if it is competitive, if it generates relatively more electricity for less cost ( a lower cost to energy ratio - $/mmbtu) : the renewable fuel is nuclear since it is technically replenished by geologic processes. 
* Using a time series plot to visualise fuel type over time : **Fuel_type_code_pudl** vs **Report_year**



###**The Data is first imported, then cleaned, statistically explored, analysed, visualised and insights drawn to achieve the set goals.**

In [ ]:
#Importing all necessary libraries 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Import the the data set
url = 'https://raw.githubusercontent.com/WalePhenomenon/climate_change/master/fuel_ferc1.csv'

fuel_data = pd.read_csv(url, error_bad_lines = False)

### Data Cleaning. This will involve:

*   Check for missed data in the data set and filling them if possible, else, drop them.
*   Check for Duplicates.
*   Check to confirm that each entry in a column is of the right data type.




In [ ]:
#Confirming if any of the columns contain missing values
fuel_data.isnull().sum()

#result shows that fuel type column contains 180 missing data.b

record_id                         0
utility_id_ferc1                  0
report_year                       0
plant_name_ferc1                  0
fuel_type_code_pudl               0
fuel_unit                       180
fuel_qty_burned                   0
fuel_mmbtu_per_unit               0
fuel_cost_per_unit_burned         0
fuel_cost_per_unit_delivered      0
fuel_cost_per_mmbtu               0
dtype: int64

In [ ]:
#To check what type of data each column is, especially the fuel_data column to understand what to do with the NA
fuel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29523 entries, 0 to 29522
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   record_id                     29523 non-null  object 
 1   utility_id_ferc1              29523 non-null  int64  
 2   report_year                   29523 non-null  int64  
 3   plant_name_ferc1              29523 non-null  object 
 4   fuel_type_code_pudl           29523 non-null  object 
 5   fuel_unit                     29343 non-null  object 
 6   fuel_qty_burned               29523 non-null  float64
 7   fuel_mmbtu_per_unit           29523 non-null  float64
 8   fuel_cost_per_unit_burned     29523 non-null  float64
 9   fuel_cost_per_unit_delivered  29523 non-null  float64
 10  fuel_cost_per_mmbtu           29523 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 2.5+ MB


In [ ]:
# I will use the mode of the fuel_data column to fill the missing values, other methods were stated in the further readings...

fuel_data.describe( include = 'all')

 


,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
count,29523,29523.000000,29523.000000,29523,29523,29343,2.952300e+04,29523.000000,29523.000000,2.952300e+04,29523.000000
unique,29523,NaN,NaN,2315,6,9,NaN,NaN,NaN,NaN,NaN
top,f1_fuel_2003_12_164_0_13,NaN,NaN,big stone,gas,mcf,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,156,11486,11354,NaN,NaN,NaN,NaN,NaN
mean,NaN,118.601836,2005.806050,NaN,NaN,NaN,2.622119e+06,8.492111,208.649031,9.175704e+02,19.304354
std,NaN,74.178353,7.025483,NaN,NaN,NaN,9.118004e+06,10.600220,2854.490090,6.877593e+04,2091.540939
min,NaN,1.000000,1994.000000,NaN,NaN,NaN,1.000000e+00,0.000001,-276.080000,-8.749370e+02,-41.501000
25%,NaN,55.000000,2000.000000,NaN,NaN,NaN,1.381700e+04,1.024000,5.207000,3.778500e+00,1.940000
50%,NaN,122.000000,2006.000000,NaN,NaN,NaN,2.533220e+05,5.762694,26.000000,1.737100e+01,4.127000
75%,NaN,176.000000,2012.000000,NaN,NaN,NaN,1.424034e+06,17.006000,47.113000,4.213700e+01,7.745000


## Remarks on the statistical description
* The record ID is unique to each entry. The utility ID has a range of 1 to 514.
* The year covers from 1994 to 2018. Since median(50%) is true then that column is correct.
* It appears most of the data was collected from big stone.
* Gas was used the most(11486).  It has 6 unique entry my love.
* The fuel unit has 9 entries with mcf with highest entry.
* The fuel_qty_burned has a minimum value of 1.
* The fuel_mmbtu_per_unit has a minimum value of 0.000001.
* The other costs have a minimum negative value, this could suggest that no cost was incurred or maybe it was an error. 

#### From the last insights I will look up that row.

In [ ]:
fuel_data[fuel_data['fuel_cost_per_unit_burned'] == -276.080000	]

#From the results, I am not to sure let me look up another

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
7733,f1_fuel_2000_12_194_3_3,194,2000,nelson dewey unit 2,waste,ton,39.0,0.0155,-276.08,-174.67,-8.906


In [ ]:
fuel_data[fuel_data['fuel_cost_per_unit_delivered'] == -8.749370e+02		]

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
17525,f1_fuel_2008_12_177_1_1,177,2008,mexico c.t.,oil,bbl,811.0,5.754,86.628,-874.937,15.055


#### At this point I will consult with an energy expert to understand the implication of the negative cost.

In [ ]:
#to find out the mode
fuel_data['fuel_unit'].mode()



0    mcf
dtype: object

In [ ]:
#mcf has the highest mode, hence we will fill the missing values With the mode
fuel_data[['fuel_unit']] = fuel_data[['fuel_unit']].fillna(value='mcf')
fuel_data.groupby('fuel_unit')['fuel_unit'].count()

fuel_unit
bbl        7998
gal          84
gramsU      464
kgU         110
mcf       11534
mmbtu       180
mwdth        95
mwhth       100
ton        8958
Name: fuel_unit, dtype: int64

In [ ]:
#to confirm no missing value exist
fuel_data.isnull().sum()

record_id                       0
utility_id_ferc1                0
report_year                     0
plant_name_ferc1                0
fuel_type_code_pudl             0
fuel_unit                       0
fuel_qty_burned                 0
fuel_mmbtu_per_unit             0
fuel_cost_per_unit_burned       0
fuel_cost_per_unit_delivered    0
fuel_cost_per_mmbtu             0
dtype: int64

In [ ]:
#checking for duplicates

fuel_data.duplicated().any()

False

In [ ]:
# commonly used form of a missing entry can be ? or - we will confirm if the data set doesn't contain this



###**Since the Data set is now cleaned, we will carry out some exploratory statistics and the visualise our data and draw our insights from the results**

### The Data will have to be broken further to make sense of them.
### I will break them up based on the years 

In [ ]:
#to confirm the frequencies of the years in the dataset
fuel_data['report_year'].value_counts()

2000    1373
2001    1356
2005    1269
2007    1264
2010    1261
2006    1243
2012    1243
2011    1240
1994    1235
2008    1228
2009    1222
2003    1211
2002    1205
1995    1201
2013    1199
2004    1192
2014    1171
1998    1107
1997    1094
2015    1093
1996    1088
1999    1050
2016    1034
2017     993
2018     951
Name: report_year, dtype: int64

In [ ]:
#I started with iloc slicing method then came across conditional slicing the used that for the rest.
Year_1994 = fuel_data[fuel_data.report_year == 1994]
Year_1994['report_year'].value_counts()

1994    1235
Name: report_year, dtype: int64

In [ ]:
Year_1995 = fuel_data[fuel_data.report_year == 1995]
Year_1995['report_year'].value_counts()

1995    1201
Name: report_year, dtype: int64

In [ ]:
Year_1996 = fuel_data[fuel_data.report_year == 1996]
Year_1996['report_year'].value_counts()

1996    1088
Name: report_year, dtype: int64

In [ ]:
Year_1997 = fuel_data[fuel_data.report_year == 1997]
Year_1998 = fuel_data[fuel_data.report_year == 1998]
Year_1999 = fuel_data[fuel_data.report_year == 1999]
Year_2000 = fuel_data[fuel_data.report_year == 2000]
Year_2001 = fuel_data[fuel_data.report_year == 2001]
Year_2002 = fuel_data[fuel_data.report_year == 2002]
Year_2003 = fuel_data[fuel_data.report_year == 2003]
Year_2004 = fuel_data[fuel_data.report_year == 2004]
Year_2005 = fuel_data[fuel_data.report_year == 2005]
Year_2006 = fuel_data[fuel_data.report_year == 2006]
Year_2007 = fuel_data[fuel_data.report_year == 2007]
Year_2008 = fuel_data[fuel_data.report_year == 2008]
Year_2009 = fuel_data[fuel_data.report_year == 2009]
Year_2010 = fuel_data[fuel_data.report_year == 2010]
Year_2011 = fuel_data[fuel_data.report_year == 2011]
Year_2012 = fuel_data[fuel_data.report_year == 2012]
Year_2013 = fuel_data[fuel_data.report_year == 2013]
Year_2014 = fuel_data[fuel_data.report_year == 2014]
Year_2015 = fuel_data[fuel_data.report_year == 2015]
Year_2016 = fuel_data[fuel_data.report_year == 2016]
Year_2017 = fuel_data[fuel_data.report_year == 2017]
Year_2018 = fuel_data[fuel_data.report_year == 2018]

#repeating all the codes since prior to this cell it has been correct 


#To get the the counts of fuel type in each year and then visualise this




In [ ]:
print(Year_1994['fuel_type_code_pudl'].value_counts())
print(Year_1995['fuel_type_code_pudl'].value_counts())
print(Year_1996['fuel_type_code_pudl'].value_counts())
print(Year_1997['fuel_type_code_pudl'].value_counts())
print(Year_1998['fuel_type_code_pudl'].value_counts())
print(Year_1999['fuel_type_code_pudl'].value_counts())
print(Year_2000['fuel_type_code_pudl'].value_counts())
print(Year_2001['fuel_type_code_pudl'].value_counts())
print(Year_2002['fuel_type_code_pudl'].value_counts())
print(Year_2003['fuel_type_code_pudl'].value_counts())
print(Year_2004['fuel_type_code_pudl'].value_counts())
print(Year_2005['fuel_type_code_pudl'].value_counts())
print(Year_2006['fuel_type_code_pudl'].value_counts())
print(Year_2007['fuel_type_code_pudl'].value_counts())
print(Year_2008['fuel_type_code_pudl'].value_counts())
print(Year_2009['fuel_type_code_pudl'].value_counts())
print(Year_2010['fuel_type_code_pudl'].value_counts())
print(Year_2011['fuel_type_code_pudl'].value_counts())
print(Year_2012['fuel_type_code_pudl'].value_counts())
print(Year_2013['fuel_type_code_pudl'].value_counts())
print(Year_2014['fuel_type_code_pudl'].value_counts())
print(Year_2015['fuel_type_code_pudl'].value_counts())
print(Year_2016['fuel_type_code_pudl'].value_counts())
print(Year_2017['fuel_type_code_pudl'].value_counts())
print(Year_2018['fuel_type_code_pudl'].value_counts())

gas        487
coal       475
oil        166
nuclear     71
waste       24
other       12
Name: fuel_type_code_pudl, dtype: int64
gas        503
coal       475
oil        113
nuclear     73
waste       22
other       15
Name: fuel_type_code_pudl, dtype: int64
coal       431
gas        426
oil        160
nuclear     41
waste       22
other        8
Name: fuel_type_code_pudl, dtype: int64
gas        448
coal       417
oil        163
nuclear     31
waste       23
other       12
Name: fuel_type_code_pudl, dtype: int64
gas        515
coal       431
oil         68
nuclear     57
waste       20
other       16
Name: fuel_type_code_pudl, dtype: int64
gas        493
coal       425
nuclear     57
oil         48
waste       14
other       13
Name: fuel_type_code_pudl, dtype: int64
gas        474
oil        416
coal       403
nuclear     56
waste       14
other       10
Name: fuel_type_code_pudl, dtype: int64
oil        474
gas        454
coal       363
nuclear     44
waste       15
other        6


In [ ]:
# To answer the first objective, I will be using using the fuel cost and comparing it to the plants.

fuel_data[fuel_data.plant_name_ferc1 == 'big stone']

# Checking the relationship for 2315 individual plants not so feasible based on my present knowledge 

# I will just used big stone as a case study

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
553,f1_fuel_1994_12_132_0_4,132,1994,big stone,coal,bbl,1290598.0,12.098000,13.290,13.220,1.099
554,f1_fuel_1994_12_132_0_6,132,1994,big stone,other,ton,19361.0,0.013535,18.980,18.980,0.701
555,f1_fuel_1994_12_132_0_7,132,1994,big stone,waste,ton,3632.0,0.006990,7.220,7.220,0.516
963,f1_fuel_1994_12_95_0_10,95,1994,big stone,coal,ton,520261.0,12.098000,13.140,12.940,1.090
964,f1_fuel_1994_12_95_0_12,95,1994,big stone,waste,ton,8148.0,0.013615,19.000,18.990,0.700
...,...,...,...,...,...,...,...,...,...,...,...
28572,f1_fuel_2018_12_122_0_2,122,2018,big stone,oil,bbl,2344.0,5.880000,63.123,63.123,10.735
28766,f1_fuel_2018_12_95_0_10,95,2018,big stone,coal,ton,327876.0,16.476000,31.968,30.771,1.940
28767,f1_fuel_2018_12_95_0_11,95,2018,big stone,oil,bbl,66109.0,5.880000,91.209,103.022,15.512
29043,f1_fuel_2018_12_132_0_4,132,2018,big stone,coal,ton,850025.0,16.476000,31.968,30.771,1.940


In [ ]:
#for big stone

data_bigstone_plant = fuel_data[fuel_data.plant_name_ferc1 == 'big stone']
fuelcost_bigstone_burned = data_bigstone_plant['fuel_cost_per_unit_burned']
fuelcost_bigstone_burned.sum()

#this is the fuel cost spent for burning the kind of fuel used by big stone from 1994-2018 


6357.478

In [ ]:
#But if we wanted to investigate how much was spent by Big Stone to burn oil from 2000 - 2010

fuelcost_bigstone_burned_oil = data_bigstone_plant[data_bigstone_plant.fuel_type_code_pudl == 'oil']
fuelcost_bigstone_burned_oil_2000_2010 = fuelcost_bigstone_burned_oil[(fuelcost_bigstone_burned_oil.report_year >= 2000) & (fuelcost_bigstone_burned_oil.report_year <= 2010) ]
fuelcost_bigstone_burned_oil_2000_2010['fuel_cost_per_unit_burned'].sum()

1526.6759999999997

In [ ]:
#To investigate the fuel_types and how they influence cost 
#We all insolate the datasets for each of the fuel types and then sum each cost and visualise each

fueldata_gas = fuel_data[fuel_data.fuel_type_code_pudl == 'gas']
fueldata_coal = fuel_data[fuel_data.fuel_type_code_pudl == 'coal']
fueldata_oil = fuel_data[fuel_data.fuel_type_code_pudl == 'oil']
fueldata_nuclear = fuel_data[fuel_data.fuel_type_code_pudl == 'nuclear']
fueldata_waste = fuel_data[fuel_data.fuel_type_code_pudl == 'waste']
fueldata_other = fuel_data[fuel_data.fuel_type_code_pudl == 'other']


In [ ]:
#for the gas
gb = fueldata_gas['fuel_cost_per_unit_burned'].sum()   # total cost of burning gas
gd = fueldata_gas['fuel_cost_per_unit_delivered'].sum() # total cost of burning gas 
gm = fueldata_gas['fuel_cost_per_mmbtu'].sum()            # total cost of energy generated
gascosts = pd.Series([gb,gd,gm])
gascosts

0    156891.82900
1    138925.15000
2     62126.34574
dtype: float64

In [ ]:
#for the coal 
cb = fueldata_coal['fuel_cost_per_unit_burned'].sum()   # total cost of burning
cd = fueldata_coal['fuel_cost_per_unit_delivered'].sum() # total cost of burning
cm = fueldata_coal['fuel_cost_per_mmbtu'].sum()            # total cost of energy generated
coalcosts = pd.Series([cb,cd,cm])

#for the oil 
ob = fueldata_oil['fuel_cost_per_unit_burned'].sum()   # total cost of burning
od = fueldata_oil['fuel_cost_per_unit_delivered'].sum() # total cost of burning
om = fueldata_oil['fuel_cost_per_mmbtu'].sum()            # total cost of energy generated
oilcosts = pd.Series([ob,od,om])

#for the nuclear
nb = fueldata_nuclear['fuel_cost_per_unit_burned'].sum()   # total cost of burning
nd = fueldata_nuclear['fuel_cost_per_unit_delivered'].sum() # total cost of burning
nm = fueldata_nuclear['fuel_cost_per_mmbtu'].sum()            # total cost of energy generated
nuclearcosts = pd.DataFrame([nb,nd,nm])

#for the waste 
wb = fueldata_waste['fuel_cost_per_unit_burned'].sum()   # total cost of burning
wd = fueldata_waste['fuel_cost_per_unit_delivered'].sum() # total cost of burning
wm = fueldata_waste['fuel_cost_per_mmbtu'].sum()            # total cost of energy generated
wastecosts = pd.Series([wb,wd,wm])

#for the others 
otb = fueldata_other['fuel_cost_per_unit_burned'].sum()   # total cost of burning
otd = fueldata_other['fuel_cost_per_unit_delivered'].sum() # total cost of burning
otm = fueldata_other['fuel_cost_per_mmbtu'].sum()            # total cost of energy generated
otherscosts = pd.Series([otb,otd,otm])


###**Data Visualization**





In [ ]:
#all necassary libraries have been imported.So we can see the plots in the workbook directly
 %matplotlib inline

 



# The Stage A Quiz


In [ ]:
# Prior to the Quiz the Data has been cleaned: checked for null values and replaced, checked for duplicates and statistically explored the data.
# The Headings above explained each step

fuel_data

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
0,f1_fuel_1994_12_1_0_7,1,1994,rockport,coal,ton,5377489.0,16.590,18.59,18.53,1.121
1,f1_fuel_1994_12_1_0_10,1,1994,rockport total plant,coal,ton,10486945.0,16.592,18.58,18.53,1.120
2,f1_fuel_1994_12_2_0_1,2,1994,gorgas,coal,ton,2978683.0,24.130,39.72,38.12,1.650
3,f1_fuel_1994_12_2_0_7,2,1994,barry,coal,ton,3739484.0,23.950,47.21,45.99,1.970
4,f1_fuel_1994_12_2_0_10,2,1994,chickasaw,gas,mcf,40533.0,1.000,2.77,2.77,2.570
...,...,...,...,...,...,...,...,...,...,...,...
29518,f1_fuel_2018_12_12_0_13,12,2018,neil simpson ct #1,gas,mcf,18799.0,1.059,4.78,4.78,9.030
29519,f1_fuel_2018_12_12_1_1,12,2018,cheyenne prairie 58%,gas,mcf,806730.0,1.050,3.65,3.65,6.950
29520,f1_fuel_2018_12_12_1_10,12,2018,lange ct facility,gas,mcf,104554.0,1.060,4.77,4.77,8.990
29521,f1_fuel_2018_12_12_1_13,12,2018,wygen 3 bhp 52%,coal,ton,315945.0,16.108,3.06,14.76,1.110


In [ ]:
#confirming we have no null values and duplicates

fuel_data.isnull().sum()

#None exists

fuel_data.duplicated().any()

#none exists

False

#QUIZ'S CODES 

In [ ]:
# Question 3

fuel_data.groupby('fuel_type_code_pudl').fuel_cost_per_unit_burned.mean()  # group by fuel type and the call up the column of fuel cost per unit burned and find the mean

# Answer is gas 

fuel_type_code_pudl
coal         67.421830
gas          13.659397
nuclear    4955.157002
oil         168.877086
other        18.253856
waste        19.518122
Name: fuel_cost_per_unit_burned, dtype: float64

In [ ]:
# Question 4

fuel_data['fuel_mmbtu_per_unit'].describe()      # The describe function contains the STD and percentile. Specifying the column

# Answer is std is 10.60 and 75% percentile is  17.01

count    29523.000000
mean         8.492111
std         10.600220
min          0.000001
25%          1.024000
50%          5.762694
75%         17.006000
max        341.260000
Name: fuel_mmbtu_per_unit, dtype: float64

In [ ]:
# Question 5

fuel_data['fuel_qty_burned'].skew()             # skewness for the required column

# answer is 15.85


fuel_data['fuel_qty_burned'].kurtosis()       #kurtosisi for the required column

#answer is 651.37


651.3694501337732

In [ ]:
#Question 6 has been addressed in the cleaning part of the code.
fuel_data.shape

# to calculate the percent 

(180/29523)*100


#answer is 0.610 %

0.609694136774718

In [ ]:
#Question 7

# the feature is a string and i will use mode manipulation to fill it as i did in the cleaning of the dataset above.

In [ ]:
# To check which has the lowest correlation 
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['record_id'])
# The error implies they both have to be numeric values.

TypeError: ignored

In [ ]:
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['utility_id_ferc1'])

-0.0378627075999726

In [ ]:
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['report_year'])

0.013599134438134116

In [ ]:
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['fuel_qty_burned'])	 

-0.018535416794825225

In [ ]:
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['fuel_mmbtu_per_unit'])	 

-0.010033808428853071

In [ ]:
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['fuel_cost_per_unit_delivered'])	 

0.01100740329916511

In [ ]:
fuel_data['fuel_cost_per_unit_burned'].corr(fuel_data['fuel_cost_per_mmbtu'])	 

-0.0004372104439704003

In [ ]:
a = [-0.0378627075999726, 0.013599134438134116, -0.018535416794825225,0.013599134438134116,-0.010033808428853071, 0.01100740329916511, -0.0004372104439704003 ]
#using sort to arrange them

In [ ]:
#Question 10
fuel_data.groupby('report_year').fuel_cost_per_unit_delivered.mean() # group by year and the call up the column of fuel cost per unit delivered and find the mean and check the max


# 1997 I used max function to confirm

report_year
1994       63.636060
1995       32.735269
1996     9196.705948
1997    11140.197239
1998      287.154420
1999       25.551627
2000      985.362877
2001       60.050396
2002       47.594361
2003       55.663493
2004      139.524275
2005       41.438184
2006       38.657484
2007       43.325023
2008       58.588197
2009      652.694163
2010       91.862105
2011       59.774667
2012       60.994502
2013      172.307591
2014      192.737183
2015      326.535511
2016      103.901761
2017       46.196861
2018      499.269966
Name: fuel_cost_per_unit_delivered, dtype: float64

In [ ]:
#Question 9

df1 = fuel_data.iloc[0:,2]      #The Report Year Column
df2 = fuel_data.iloc[0:,4]      #The Fuel Type Column
df3 = fuel_data.iloc[0:,6]      # The Fuel Qty Burned

Df = pd.concat([df1,df2,df3], axis = 1)       #The Combination of all three columns
print(Df)

       report_year fuel_type_code_pudl  fuel_qty_burned
0             1994                coal        5377489.0
1             1994                coal       10486945.0
2             1994                coal        2978683.0
3             1994                coal        3739484.0
4             1994                 gas          40533.0
...            ...                 ...              ...
29518         2018                 gas          18799.0
29519         2018                 gas         806730.0
29520         2018                 gas         104554.0
29521         2018                coal         315945.0
29522         2018                 gas          17853.0

[29523 rows x 3 columns]


In [ ]:
Df_1994 = Df[Df.report_year == 1994]        # For the year 1994
Df_1998 = Df[Df.report_year == 1998]        # For the year 1998

In [ ]:
Df_1994.groupby('fuel_type_code_pudl').fuel_qty_burned.sum()        #Grouping by fuel type and then summing the fuel burned for each fuel type for 1994


#The Value of interest is 7.172082e+08

fuel_type_code_pudl
coal       7.172082e+08
gas        2.432212e+09
nuclear    2.069366e+08
oil        1.469716e+06
other      5.094458e+06
waste      1.242241e+06
Name: fuel_qty_burned, dtype: float64

In [ ]:
Df_1998.groupby('fuel_type_code_pudl').fuel_qty_burned.sum()         #Grouping by fuel type and then summing the fuel burned for each fuel type for 1994


#The value of interest is  8.982095e+08

fuel_type_code_pudl
coal       8.982095e+08
gas        2.554651e+09
nuclear    1.504636e+08
oil        5.436080e+05
other      1.157796e+07
waste      1.624599e+06
Name: fuel_qty_burned, dtype: float64

In [ ]:
a = (((7.172082e+08)-(8.982095e+08))/8.982095e+08) * 100  # using 1998 as original year and 1994 as new year: the formula is (change/initial)*100

print(a)
#Answer is -20.15% is -20%


-20.151345537984177


#### I think the format of answer of question of question 6 is not clear and the question 9 was not so correct, as you can use from my calculation it shows the answer is -20%... regardless of this, the quiz was stimulating and very relevant. Thank you so much Team Hamoye.